In [1]:
# import the following libraries into the Python environment

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [2]:
# Load the dataset
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv', encoding='latin-1')

In [3]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
# Selecting only the 'tweet_text' and 'is_there_an_emotion_directed_at_a_brand_or_product' columns for sentiment analysis

data=data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]

In [5]:
# Rename columns for simplicity

data.columns = ['tweet', 'sentiment']

In [6]:
data.shape

(9093, 2)

In [8]:
# Display full content of DataFrame columns

pd.set_option('display.max_colwidth',None)

In [9]:
 data.head()

,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [10]:
data['sentiment'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: sentiment, dtype: int64

# Preprocess the data

In [11]:
# Count the number of missing values in the 'tweet' column

data['tweet'].isnull().sum()

1

In [12]:
# handle missing values

data['tweet'].fillna('', inplace=True)

In [13]:
# Extracting input features (x) and target labels (y) from the dataset

x = data['tweet']
y = data['sentiment']

In [14]:
y

0                         Negative emotion
1                         Positive emotion
2                         Positive emotion
3                         Negative emotion
4                         Positive emotion
                       ...                
9088                      Positive emotion
9089    No emotion toward brand or product
9090    No emotion toward brand or product
9091    No emotion toward brand or product
9092    No emotion toward brand or product
Name: sentiment, Length: 9093, dtype: object

In [15]:
# Encode the target variable

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)


In [16]:
y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [17]:
# Tokenize the text data using Keras Tokenizer

from keras.preprocessing import text
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(list(data['tweet']))
tokenized_text=tokenizer.texts_to_sequences(data['tweet'])

In [19]:
len(tokenized_text[0])

24

In [20]:
len(tokenized_text[1])

22

In [21]:
# Pad the tokenized_text to make all text sequences the same length (100)

from keras.utils import pad_sequences
x=pad_sequences(tokenized_text,maxlen=100)

In [22]:
# Split the data into training and testing sets

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# SimpleRNN

In [24]:
# Define a sequential model with Embedding, SimpleRNN, Dropout, Dense, and Softmax layers

from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SimpleRNN,Dropout

In [23]:
len(tokenizer.word_index)

10147

In [36]:
# Create a Sequential model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))

# Add a SimpleRNN layer with 32 units

model.add(SimpleRNN(32))
model.add(Dropout(0.5))

# Add a Dense layer with 50 units
#model.add(Dense(50,activation = 'relu'))
#model.add(Dropout(0.5))

# Add the final Dense layer with 4 units (for 4 classes)
model.add(Dense(4, activation='softmax'))

In [37]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 128)          1298944   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                5152      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 4)                 132       
                                                                 
Total params: 1304228 (4.98 MB)
Trainable params: 1304228 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 6s 23ms/step - loss: 0.9976 - accuracy: 0.5574 - val_loss: 0.8628 - val_accuracy: 0.6168
Epoch 2/10
205/205 [==============================] - 5s 27ms/step - loss: 0.7418 - accuracy: 0.7116 - val_loss: 0.8576 - val_accuracy: 0.6332
Epoch 3/10
205/205 [==============================] - 5s 22ms/step - loss: 0.4661 - accuracy: 0.8393 - val_loss: 1.0099 - val_accuracy: 0.5934
Epoch 4/10
205/205 [==============================] - 5s 23ms/step - loss: 0.3293 - accuracy: 0.8943 - val_loss: 1.0855 - val_accuracy: 0.5907
Epoch 5/10
205/205 [==============================] - 5s 26ms/step - loss: 0.2565 - accuracy: 0.9154 - val_loss: 1.1469 - val_accuracy: 0.6140
Epoch 6/10
205/205 [==============================] - 5s 22ms/step - loss: 0.2261 - accuracy: 0.9229 - val_loss: 1.2393 - val_accuracy: 0.6044
Epoch 7/10
205/205 [==============================] - 5s 27ms/step - loss: 0.1961 - accuracy: 0.9317 - val_loss: 1.3054 - val_accuracy: 0.6044

In [40]:
y_pred=model.predict(x_test)

57/57 [==============================] - 0s 5ms/step


In [58]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 0s 5ms/step - loss: 1.4256 - accuracy: 0.6069
Test Accuracy: 60.69%


In [59]:
thresholds = [0.25, 0.5, 0.75]  # Adjust these thresholds

# Initialize an array with zeros for the predicted labels
y_pred_classes = np.zeros_like(y_pred, dtype=int)

# Assign class labels based on thresholds
for i, threshold in enumerate(thresholds):
    y_pred_classes[:, i] = (y_pred[:, i] > threshold).astype(int)

In [60]:
from sklearn.metrics import jaccard_score


jaccard_similarity = jaccard_score(y_test, y_pred_classes, average='weighted')
print(f'Jaccard Similarity: {jaccard_similarity * 100:.2f}%')

Jaccard Similarity: 30.03%


# LSTM

In [61]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))

model.add(LSTM(32))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 11s 39ms/step - loss: 0.9224 - accuracy: 0.6046 - val_loss: 0.8260 - val_accuracy: 0.6209
Epoch 2/10
205/205 [==============================] - 9s 45ms/step - loss: 0.6670 - accuracy: 0.7363 - val_loss: 0.8222 - val_accuracy: 0.6731
Epoch 3/10
205/205 [==============================] - 8s 41ms/step - loss: 0.4691 - accuracy: 0.8248 - val_loss: 0.8755 - val_accuracy: 0.6841
Epoch 4/10
205/205 [==============================] - 8s 38ms/step - loss: 0.3589 - accuracy: 0.8703 - val_loss: 0.9215 - val_accuracy: 0.6731
Epoch 5/10
205/205 [==============================] - 9s 46ms/step - loss: 0.2913 - accuracy: 0.8925 - val_loss: 0.9902 - val_accuracy: 0.6552
Epoch 6/10
205/205 [==============================] - 9s 45ms/step - loss: 0.2470 - accuracy: 0.9047 - val_loss: 1.0778 - val_accuracy: 0.6662
Epoch 7/10
205/205 [==============================] - 8s 39ms/step - loss: 0.2086 - accuracy: 0.9125 - val_loss: 1.1870 - val_accuracy: 0.664

In [62]:
y_pred=model.predict(x_test)

57/57 [==============================] - 1s 7ms/step


In [63]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 1s 14ms/step - loss: 1.4354 - accuracy: 0.6564
Test Accuracy: 65.64%


In [64]:
thresholds = [0.25, 0.5, 0.75]  # Adjust these thresholds

# Initialize an array with zeros for the predicted labels
y_pred_classes = np.zeros_like(y_pred, dtype=int)

# Assign class labels based on thresholds
for i, threshold in enumerate(thresholds):
    y_pred_classes[:, i] = (y_pred[:, i] > threshold).astype(int)

In [65]:
from sklearn.metrics import jaccard_score


jaccard_similarity = jaccard_score(y_test, y_pred_classes, average='weighted')
print(f'Jaccard Similarity: {jaccard_similarity * 100:.2f}%')

Jaccard Similarity: 34.45%


## Implement early stopping to prevent overfitting

In [66]:
#early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Modify your model.fit call to include the early_stopping callback
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
182/182 [==============================] - 8s 44ms/step - loss: 0.1357 - accuracy: 0.9361 - val_loss: 0.8133 - val_accuracy: 0.7890
Epoch 2/10
182/182 [==============================] - 7s 38ms/step - loss: 0.1285 - accuracy: 0.9411 - val_loss: 0.9027 - val_accuracy: 0.7849
Epoch 3/10
182/182 [==============================] - 8s 43ms/step - loss: 0.1158 - accuracy: 0.9405 - val_loss: 0.9051 - val_accuracy: 0.7849
Epoch 4/10
182/182 [==============================] - 8s 41ms/step - loss: 0.1081 - accuracy: 0.9459 - val_loss: 1.0343 - val_accuracy: 0.7842


In [70]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 0s 7ms/step - loss: 1.5090 - accuracy: 0.6427
Test Accuracy: 64.27%
